### Scrape Wikipedia page, parse table of postal codes and neighborhoods, reformat entries and create Pandas dataframe.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source,'lxml')

In [4]:
table = soup.table 

In [5]:
table_body = table.find('tbody')

In [6]:
rows = table_body.find_all('tr')

So far, we have used BeautifulSoup to parse the source code from the Wikipedia page on the html 'tbody' tag, and then parsed on the 'tr' tag which defines table rows.

In [7]:
data = []
for row in rows:
    col = row.find_all('td')
    col = [e.text.strip() for e in col]
    data.append([e for e in col if e])
data = [item for item in data if item]

In [8]:
data = [item for item in data if item[1] != 'Not assigned']

The above codeblock loops over each row of the table and parses on the 'td' tag, then strips all but the text string and appends to a list. This creates a list where each entry is a row of the table. We then use a list comprehension to filter out empty lists (first entry was empty, for instance). Finally, we access the second index of each item in the list and filter out items where index 1 has the string value 'Not assigned'; this is to remove table entries where the Borough is not assigned, as instructed. 

In [9]:
df = pd.DataFrame(data)

In [10]:
names = ['Postal code','Borough','Neighborhood']
df.columns = names

In [11]:
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


The next step is to replace all instances of slash (/) separators with commas(,). After this, we check if any dataframe entries have assigned Boroughs but unassigned Neighborhoods, and if so we will assign the Neighborhood to be the same as the Borough.

In [12]:
df['Neighborhood'] = df.apply(lambda row: row['Neighborhood'].replace(' /',','),axis=1)
df[df['Neighborhood'] == "Not assigned"]

,Postal code,Borough,Neighborhood


In [13]:
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
print("Number of rows: " + str(df.shape[0]))

Number of rows: 103


### END PART ONE OF ASSIGNMENT

## Load postal code lat/long CSV and merge with dataframe

In [15]:
coords = pd.read_csv(r'C:\Users\Turcanhydgoongod\Documents\Jupyter\Coursera-PythonForData\Geospatial_Coordinates.csv')

In [16]:
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We can sort dataframes by the Postal Code and then join on that key

In [17]:
df.rename(columns={'Postal code':'Postal Code'},inplace=True)
coords.sort_values(by='Postal Code',inplace=True)
df.sort_values(by='Postal Code',inplace=True)


In [18]:
tor = pd.merge(df,coords,on='Postal Code')

In [19]:
tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


We now have a single dataframe consisting of a postal code, borough, neighborhood, and lat/long coordinates. 

### END PART TWO OF ASSIGNMENT

In [20]:
latitude = tor['Latitude'].mean(axis=0)
longitude = tor['Longitude'].mean(axis=0)

In [21]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], tor['Borough'], tor['Neighborhood']):
    label = '{}; {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Next, we will access Foursquare's API to get more information about each neighborhood.

In [22]:
CLIENT_ID = 'VH2J1S1NRIEE0YPB5C4Q3SCS2LE4QL0WNYD01H1PQ43PY2EA'
CLIENT_SECRET = 'OC21X3LN5TD42N1MMQ0VYKQ42RE5WI32FIWFQZBKDOMX4BZZ'
VERSION = '20200303'

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: VH2J1S1NRIEE0YPB5C4Q3SCS2LE4QL0WNYD01H1PQ43PY2EA
CLIENT_SECRET:OC21X3LN5TD42N1MMQ0VYKQ42RE5WI32FIWFQZBKDOMX4BZZ


In [23]:
neighborhood_latitude = tor.iloc[0][3]
neighborhood_longitude = tor.iloc[0][4]

In [24]:
LIMIT = 150
radius = 750

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8972ed3907e7001bb86112'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.81343630675, 'lng': -79.18501767109562},
   'sw': {'lat': 43.79993629324999, 'lng': -79.2036891289044}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '579a91b3498e9bd833afa78a',
       'name': "Wendy's",
       'location': {'address': '8129 Sheppard Avenue',
        'lat': 43.8020084,
        'lng': -79.1980797,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.8020084,
          'lng': -79.1980797}],
        'distance': 600,
        'postalCode': 'M1B 6A3',
        'cc': 'CA',
        'city': 'S

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
pd.json_normalize(results)

,meta.code,meta.requestId,response.headerLocation,response.headerFullLocation,response.headerLocationGranularity,response.totalResults,response.suggestedBounds.ne.lat,response.suggestedBounds.ne.lng,response.suggestedBounds.sw.lat,response.suggestedBounds.sw.lng,response.groups
0,200,5e8972ed3907e7001bb86112,Malvern,"Malvern, Toronto",neighborhood,10,43.813436,-79.185018,43.799936,-79.203689,"[{'type': 'Recommended Places', 'name': 'recom..."


In [28]:
def getCatType(row):
    try:
        return row['categories'][0].get('name')
    except:
        return row['venue.categories'][0].get('name')

In [29]:
def getCatType(row):
    try:
        return row['categories'][0].get('name')
    except KeyError:
        pass
    try:
        return row['venue.categories'][0].get('name')
    except:
        print(row)

In [30]:
venues = results['response']['groups'][0]['items']
venues = pd.json_normalize(venues)


In [31]:
venues['venue.categories'] = venues.apply(getCatType,axis=1)

In [32]:
venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.venuePage.id'],
      dtype='object')

In [33]:
venues_to_keep = ['venue.name','venue.location.lat','venue.location.lng','venue.categories']
venues = venues[venues_to_keep]
new_names = ['Venue Name','Venue Location Lat','Venue Location Long','Venue Categories']
rename_dict = dict(zip(venues_to_keep, new_names))
venues = venues.rename(rename_dict,axis=1)

In [34]:
venues

,Venue Name,Venue Location Lat,Venue Location Long,Venue Categories
0,Wendy's,43.802008,-79.198080,Fast Food Restaurant
1,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
2,Tim Hortons,43.802000,-79.198169,Coffee Shop
3,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
4,Lee Valley,43.803161,-79.199681,Hobby Shop
5,Images Salon & Spa,43.802283,-79.198565,Spa
6,FASTSIGNS,43.807882,-79.201968,Business Service
7,Ecopainting inc.,43.808417,-79.202392,Construction & Landscaping
8,Bella's Lechon,43.801291,-79.198378,Filipino Restaurant
9,Rouge Park - Woodland Trail,43.801782,-79.200427,Trail


### After this proof of concept, we write a function that will loop over our list of neighborhoods and build a data frame of venues

In [39]:
def buildVenuesDataframe(df, radius=750, LIMIT=150, VERSION = '20200303'):
    venue_df = pd.DataFrame()
    new_names = ['Venue Name','Venue Lat','Venue Long','Venue Categories']
    venues_to_keep = ['venue.name','venue.location.lat','venue.location.lng','venue.categories']
    rename_dict = dict(zip(venues_to_keep, new_names))
    CLIENT_ID = 'VH2J1S1NRIEE0YPB5C4Q3SCS2LE4QL0WNYD01H1PQ43PY2EA'
    CLIENT_SECRET = 'OC21X3LN5TD42N1MMQ0VYKQ42RE5WI32FIWFQZBKDOMX4BZZ'
    
    for index, row in df.iloc[:,2:5].iterrows():
        name = row[0]
        lat = row[1]
        lng = row[2]
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = pd.json_normalize(results)
        if not venues.empty:
            venues['venue.categories'] = venues.apply(getCatType,axis=1) #apply function to extract category name from dict
            venues = venues[venues_to_keep]
            venues = venues.rename(rename_dict,axis=1)
            venues['Neighborhood'] = name
            venues['Neighborhood Lat'] = lat
            venues['Neighborhood Long'] = lng
            venue_df = venue_df.append(venues)
    return venue_df.reset_index()

In [40]:
venues_df = buildVenuesDataframe(tor)

In [41]:
venues_df.shape

(3731, 8)

In [42]:
venues_df.head()

,index,Venue Name,Venue Lat,Venue Long,Venue Categories,Neighborhood,Neighborhood Lat,Neighborhood Long
0,0,Wendy's,43.802008,-79.198080,Fast Food Restaurant,"Malvern, Rouge",43.806686,-79.194353
1,1,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,"Malvern, Rouge",43.806686,-79.194353
2,2,Tim Hortons,43.802000,-79.198169,Coffee Shop,"Malvern, Rouge",43.806686,-79.194353
3,3,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store,"Malvern, Rouge",43.806686,-79.194353
4,4,Lee Valley,43.803161,-79.199681,Hobby Shop,"Malvern, Rouge",43.806686,-79.194353


In [44]:
sum(venues_df.duplicated()) #no repeated entries, despite row 0 and 1 looking to be quite similar

0

In [49]:
venues_df.drop('index',axis=1,inplace=True)

In [50]:
venues_df.shape

(3731, 7)

In [57]:
venues_df.head()

,Venue Name,Venue Lat,Venue Long,Venue Categories,Neighborhood,Neighborhood Lat,Neighborhood Long
0,Wendy's,43.802008,-79.198080,Fast Food Restaurant,"Malvern, Rouge",43.806686,-79.194353
1,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,"Malvern, Rouge",43.806686,-79.194353
2,Tim Hortons,43.802000,-79.198169,Coffee Shop,"Malvern, Rouge",43.806686,-79.194353
3,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store,"Malvern, Rouge",43.806686,-79.194353
4,Lee Valley,43.803161,-79.199681,Hobby Shop,"Malvern, Rouge",43.806686,-79.194353


In [52]:
venues_df.to_csv("venues",index=False) #write dataframe to csv for importing later/ease of access

In [55]:
venues_df.groupby('Neighborhood').count()

,Venue Name,Venue Lat,Venue Long,Venue Categories,Neighborhood Lat,Neighborhood Long
Neighborhood,,,,,,
Agincourt,14,14,14,14,14,14
"Alderwood, Long Branch",15,15,15,15,15,15
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,9,9,9,9,9,9
"Bedford Park, Lawrence Manor East",36,36,36,36,36,36
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",8,8,8,8,8,8
Woburn,5,5,5,5,5,5
Woodbine Heights,12,12,12,12,12,12


In [61]:
print(f"There are {len(venues_df['Venue Categories'].unique())} unique venue categories")

There are 326 unique venue categories


After having issues with 1-hot encoding, it was discovered that one of the Venue Categories was 'Neighborhood', which was creating a column called 'Neighborhood' in the encoded matrix. Upon further inspection, as seen in cell below, these 'venues' are just neighborhoods that we have already defined/included in our data. Therefore we will drop those rows from venues_df, then perform the encoding

In [92]:
venues_df = venues_df[venues_df['Venue Categories'] != 'Neighborhood']

Create 1-hot encoding matrix for categorical variable 'Venue Categories'

In [94]:
toronto_onehot = pd.get_dummies(venues_df[['Venue Categories']],prefix="",prefix_sep="")

toronto_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
reorder_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[reorder_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
toronto_onehot.shape

(3723, 326)

Now we will group our dataframe by neighborhood, change the variables to be the 

In [106]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [150]:
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,...,0.0,0.0,0.027778,0.0,0.0,0.0,0.0,0.027778,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
95,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


Display top 4 venues by relative frequency for each neighborhood; .07 indicates that about 7% of venues in our neighborhood (based on our defined radius) are of the corresponding type. 

In [179]:
from IPython.display import display, HTML

In [184]:
num_top_venues = 4

for place in toronto_grouped['Neighborhood']:
    print("----"+place+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == place].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    display(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----


,venue,freq
0,Latin American Restaurant,0.07
1,Discount Store,0.07
2,Pool Hall,0.07
3,Supermarket,0.07




----Alderwood, Long Branch----


,venue,freq
0,Convenience Store,0.13
1,Pizza Place,0.13
2,Gym,0.07
3,Dance Studio,0.07




----Bathurst Manor, Wilson Heights, Downsview North----


...

,venue,freq
0,Skating Rink,0.17
1,Cosmetics Shop,0.08
2,Video Store,0.08
3,Beer Store,0.08




----York Mills West----


,venue,freq
0,Park,0.43
1,Bowling Alley,0.14
2,Convenience Store,0.14
3,Bank,0.14




----York Mills, Silver Hills----


,venue,freq
0,Pool,1.0
1,Accessories Store,0.0
2,Music Store,0.0
3,Optical Shop,0.0


In [200]:
for neighborhood in venues_df['Neighborhood'].unique():
    display("===="+neighborhood+"====",venues_df[venues_df['Neighborhood']==neighborhood]['Venue Categories'].value_counts())

'====Malvern, Rouge===='

Fast Food Restaurant             2
Spa                              1
Business Service                 1
Coffee Shop                      1
Hobby Shop                       1
Paper / Office Supplies Store    1
Construction & Landscaping       1
Trail                            1
Filipino Restaurant              1
Name: Venue Categories, dtype: int64

'====Rouge Hill, Port Union, Highland Creek===='

Breakfast Spot        2
Burger Joint          1
Bar                   1
Italian Restaurant    1
Name: Venue Categories, dtype: int64

'====Guildwood, Morningside, West Hill===='

...

Gas Station            1
Pharmacy               1
American Restaurant    1
Beer Store             1
Chinese Restaurant     1
Shopping Mall          1
Intersection           1
Coffee Shop            1
Sandwich Place         1
Pizza Place            1
Bank                   1
Supermarket            1
Mobile Phone Shop      1
Name: Venue Categories, dtype: int64

'====South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens===='

Grocery Store           3
Pizza Place             2
Fast Food Restaurant    1
Hardware Store          1
Pharmacy                1
Fried Chicken Joint     1
Gym Pool                1
Sandwich Place          1
Beer Store              1
Caribbean Restaurant    1
Name: Venue Categories, dtype: int64

'====Northwest===='

Lounge    1
Name: Venue Categories, dtype: int64

In [350]:
def top_categories_df(df):
    cat_df = pd.DataFrame()
    neighborhoods = venues_df['Neighborhood'].unique()
    for neighborhood in neighborhoods:
        top_cats = venues_df[venues_df['Neighborhood']==neighborhood]['Venue Categories'].value_counts(normalize=True).index
        d = {'Neighborhood':neighborhood}
        d.update({str(idx+1):[cat] for idx, cat in enumerate(top_cats[0:min(4,len(top_cats))])})
        cat_df = cat_df.append(pd.DataFrame(d))
    return cat_df.reset_index().drop('index',axis=1)
    

In [420]:
top_categories_by_neighborhood = top_categories_df(venues_df)

In [370]:
top_categories_by_neighborhood.head()

,Neighborhood,1,2,3,4
0,"Malvern, Rouge",Fast Food Restaurant,Spa,Business Service,Coffee Shop
1,"Rouge Hill, Port Union, Highland Creek",Breakfast Spot,Burger Joint,Bar,Italian Restaurant
2,"Guildwood, Morningside, West Hill",Fast Food Restaurant,Pizza Place,Sports Bar,Intersection
3,Woburn,Park,Coffee Shop,Business Service,NaN
4,Cedarbrae,Indian Restaurant,Coffee Shop,Bakery,Caribbean Restaurant


We have created a dataframe which finds the top 4 venue categories for each neighborhood (cols 1 - 4). If a neighborhood has fewer than 4 categories of venues, the category for that placement will be NaN. Note that, in comparison to the analysis of Manhattan data, Toronto has many neighborhoods where the top category has just 2 or 3 venues, and then next 10 top categories each have only 1 venue. This diminishes the value of using categories beyond the top few, because then inclusion in this data frame is arbitrary (either alphabetical, earlier order in original df, etc). We now proceed to cluster using k-means algorithm. 


In [375]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [377]:
# add clustering labels
top_categories_by_neighborhood.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(top_categories_by_neighborhood.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Spa,Business Service,Coffee Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Breakfast Spot,Burger Joint,Bar,Italian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Fast Food Restaurant,Pizza Place,Sports Bar,Intersection
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Park,Coffee Shop,Business Service,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Indian Restaurant,Coffee Shop,Bakery,Caribbean Restaurant


In [424]:
top_categories_by_neighborhood = top_categories_df(venues_df)
for kclusters in range(3,12):
    top_categories_by_neighborhood_temp = top_categories_by_neighborhood
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
    top_categories_by_neighborhood_temp['Cluster Labels'] = kmeans.labels_

    toronto_merged = tor

    # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
    toronto_merged = toronto_merged.join(top_categories_by_neighborhood.set_index('Neighborhood'), on='Neighborhood')
    display(toronto_merged['Cluster Labels'].value_counts())

1.0    94
2.0     7
0.0     1
Name: Cluster Labels, dtype: int64

0.0    59
1.0    41
2.0     1
3.0     1
Name: Cluster Labels, dtype: int64

0.0    80
1.0    19
2.0     1
3.0     1
4.0     1
Name: Cluster Labels, dtype: int64

0.0    52
5.0    33
1.0    14
2.0     1
3.0     1
4.0     1
Name: Cluster Labels, dtype: int64

0.0    51
5.0    33
1.0    14
2.0     1
3.0     1
4.0     1
6.0     1
Name: Cluster Labels, dtype: int64

0.0    90
4.0     6
5.0     1
7.0     1
6.0     1
2.0     1
1.0     1
3.0     1
Name: Cluster Labels, dtype: int64

1.0    56
7.0    32
8.0     7
4.0     2
2.0     1
3.0     1
6.0     1
5.0     1
0.0     1
Name: Cluster Labels, dtype: int64

1.0    56
7.0    32
8.0     6
4.0     2
2.0     1
9.0     1
3.0     1
6.0     1
5.0     1
0.0     1
Name: Cluster Labels, dtype: int64

1.0     59
5.0     32
8.0      2
4.0      2
0.0      1
7.0      1
9.0      1
6.0      1
2.0      1
10.0     1
3.0      1
Name: Cluster Labels, dtype: int64

Using k=6, we begin to see 3 distinct clusters rather than 2, so we will proceed with 6 means.

In [454]:

kclusters = 6
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
top_categories_by_neighborhood = top_categories_df(venues_df)
top_categories_by_neighborhood.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor

toronto_merged = toronto_merged.join(top_categories_by_neighborhood.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Spa,Business Service,Coffee Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,5.0,Breakfast Spot,Burger Joint,Bar,Italian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5.0,Fast Food Restaurant,Pizza Place,Sports Bar,Intersection
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.0,Park,Coffee Shop,Business Service,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Indian Restaurant,Coffee Shop,Bakery,Caribbean Restaurant


In [455]:
toronto_merged['Cluster Labels'].apply(int) #throws error, must find and remove NaN value in Cluster Label

ValueError: cannot convert float NaN to integer

In [456]:
toronto_merged['Cluster Labels'].value_counts()

0.0    52
5.0    33
1.0    14
2.0     1
3.0     1
4.0     1
Name: Cluster Labels, dtype: int64

In [457]:
toronto_merged.shape

(103, 10)

In [458]:
import numpy as np
toronto_merged[np.isnan(toronto_merged['Cluster Labels'])]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN


In [459]:
index_to_drop = toronto_merged[np.isnan(toronto_merged['Cluster Labels'])].index

In [461]:
toronto_merged.drop(index_to_drop,inplace=True)

In [462]:
toronto_merged.shape

(102, 10)

In [463]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].apply(int)

In [464]:
toronto_merged.drop('Postal Code',axis=1,inplace=True)

In [465]:
from matplotlib import cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now we will take a closer look at each cluster

In [466]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1,2,3,4
0,"Malvern, Rouge",Fast Food Restaurant,Spa,Business Service,Coffee Shop
4,Cedarbrae,Indian Restaurant,Coffee Shop,Bakery,Caribbean Restaurant
5,Scarborough Village,Ice Cream Shop,Fast Food Restaurant,Sandwich Place,Restaurant
7,"Golden Mile, Clairlea, Oakridge",Intersection,Bus Line,Bakery,Diner
8,"Cliffside, Cliffcrest, Scarborough Village West",Wings Joint,Pizza Place,Hardware Store,Burger Joint
9,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café
11,"Wexford, Maryvale",Burger Joint,Pizza Place,Middle Eastern Restaurant,Breakfast Spot
12,Agincourt,Seafood Restaurant,Pool Hall,Skating Rink,Lounge
13,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Intersection,Convenience Store,Pizza Place
14,"Milliken, Agincourt North, Steeles East, L'Amo...",Chinese Restaurant,Pizza Place,Noodle House,Shop & Service


In [467]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1,2,3,4
6,"Kennedy Park, Ionview, East Birchmount Park",Coffee Shop,Discount Store,Convenience Store,Chinese Restaurant
10,"Dorset Park, Wexford Heights, Scarborough Town...",Furniture / Home Store,Electronics Store,Indian Restaurant,Rental Service
30,Downsview,Vietnamese Restaurant,Coffee Shop,Grocery Store,Pizza Place
31,Downsview,Vietnamese Restaurant,Coffee Shop,Grocery Store,Pizza Place
32,Downsview,Vietnamese Restaurant,Coffee Shop,Grocery Store,Pizza Place
33,Downsview,Vietnamese Restaurant,Coffee Shop,Grocery Store,Pizza Place
36,Woodbine Heights,Skating Rink,Bus Line,Video Store,Pharmacy
53,"Regent Park, Harbourfront",Coffee Shop,Italian Restaurant,Pub,Restaurant
59,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Hotel,Restaurant,Café
65,"The Annex, North Midtown, Yorkville",Coffee Shop,Pizza Place,Café,Pub


In [468]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1,2,3,4
102,Northwest,Lounge,NaN,NaN,NaN


In [469]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1,2,3,4
63,Roselawn,Spa,Playground,Home Service,Pet Store


In [471]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1,2,3,4
61,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,American Restaurant


In [472]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1,2,3,4
1,"Rouge Hill, Port Union, Highland Creek",Breakfast Spot,Burger Joint,Bar,Italian Restaurant
2,"Guildwood, Morningside, West Hill",Fast Food Restaurant,Pizza Place,Sports Bar,Intersection
3,Woburn,Park,Coffee Shop,Business Service,NaN
17,Hillcrest Village,Shopping Mall,Chinese Restaurant,Bakery,Recreation Center
18,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Japanese Restaurant,Restaurant
22,Willowdale,Pizza Place,Coffee Shop,Ramen Restaurant,Sushi Restaurant
24,Willowdale,Pizza Place,Coffee Shop,Ramen Restaurant,Sushi Restaurant
26,Don Mills,Japanese Restaurant,Gym,Beer Store,Coffee Shop
27,Don Mills,Japanese Restaurant,Gym,Beer Store,Coffee Shop
38,Leaside,Coffee Shop,Furniture / Home Store,Electronics Store,Sporting Goods Shop


We have 3 larger clusters, and 3 smaller/singular clusters. This was the best our algorithm could do, as other values of k identified at most 2 clusters. Cluster 0 looks to contain more fast food restaurants than others, bot absolutely and in proportion. Cluster 1 has 4 Vietnamese restaurants, but they are all located in the same neighborhood, and the rest of cluster contains no other asian food or other restaurants (one pizza place). Cluster 5 seems to find neighborhoods that young people would be more likely to reside in, including many pizza places and coffee shops, as well as a brewery and discount store. 